## Restaurant Seat Optimization




#### Step 0: Obtain parameters from 'data.txt'



In [1]:
import json

w = l = customer_goodwill = REVENUE_GOAL = num_timeslots = randomly_generated_data = 0
marginal_revenue = {}

with open('../data.json') as f:
    data = json.load(f)
    w = data['restaurant_width']
    l = data['restaurant_length']
    customer_goodwill = data['customer_goodwill']
    REVENUE_GOAL = data['revenue_goal']
    randomly_generated_data = data['randomly_generated_data']
    max_party_size = data['max_party_size']
    for i in data['marginal_revenue']:
        marginal_revenue[int(i)] = data['marginal_revenue'][i]

#### Step 1: Import gurobipy module

In [2]:
import gurobipy as gp
from gurobipy import GRB
import random

#### Step 2: Define your model

In [3]:
m = gp.Model()

Using license file /Users/matthewfollegot/gurobi.lic
Academic license - for non-commercial use only


#### Step 3: Define your sets

In [4]:
# set of timeslots
K = 7

# set of parties at each timeslot
# since there are 7 timeslots and since this data is randomly generated, 
# the set of parties during timelsot k will be I[k].
# Prior to having randomly generated demand we had set I = 40 (40 parties) for each timeslot
if randomly_generated_data == True:
    I = [
        random.randint(10,15), 
        random.randint(17,24),
        random.randint(17,24),
        random.randint(10,15),
        random.randint(25,40),
        random.randint(25,40),
        random.randint(10,15),
    ]

else:
    print("Please allow data to be randomly generated to ensure accuracy and reliability of resutls")

#### Step 4: Define your parameters

In [5]:
#initializing the party 
party_size = {}
for k in range(K):
    for i in range(I[k]):
        party_size[i, k] = random.randint(2,max_party_size)

# the table mapping below represents
# how many tables a specific party_size would require
table_mapping = {}
for i in range(2,max_party_size + 1):
    table_mapping[i] = i // 2 + i % 2

# initializing the amount of space that each of the party sizes 
# would take up
space = {}
for k in range(K):
    for i in range(I[k]):
        num_tables = table_mapping.get(party_size[i, k])
        space[i, k] = 6 + 3 * num_tables

#### Step 5: Define your decision variables

In [6]:
# defining xik, whethere or not party i is seated at timeslot k
x = {}
for k in range(K):
    for i in range(I[k]):
        x[i,k] = m.addVar(lb=0.0, vtype=GRB.BINARY, name="x_"+str(i)+str(k))
        
# defining bk, whether or not there is a social distancing violation at timeslot k
b = {}
for k in range(K):
    b[k] = m.addVar(lb=0.0, obj=880, vtype=GRB.BINARY, name="b_"+str(k))
    
# defining the piecewise function for revenue
# model.setPWLObj()
s = {}
#defining space slack variable
for k in range(K):
    s[k] = m.addVar(lb=0.0, vtype=GRB.INTEGER, name="s_"+str(k))

#defining space excess variable
e = {}
for k in range(K):
    e[k] = m.addVar(lb=0.0, vtype=GRB.INTEGER, name="e_" + str(k))

u = {}
#defining seating everyone slack variable
for k in range(K):
    u[k] = m.addVar(lb=0.0, obj=customer_goodwill, vtype=GRB.INTEGER, name="u_"+str(k))

#### Step 6: Set your objective function

In [7]:
m.modelSense = GRB.MINIMIZE

#### Step 7: Add Constraints

In [8]:
# defining space constraint
for k in range(K):
    running_sum = 0
    for i in range(I[k]):
        #print(running_sum)
        #xik = m.getVarByName("x_"+str(i) + str(k)).X
        #print(xik)
#         spaceik = space[i,k]
        running_sum += x[i,k]*space[i,k]
#    print(running_sum)
    m.addConstr((s[k] - e[k] + running_sum), GRB.EQUAL, l*w)

print(f"l={l}, w={w}")
    
# defining the constraint that turns on the social distancing violated binary var
for k in range(K):
    m.addConstr(10**6*b[k], GRB.GREATER_EQUAL, e[k])

# defining the constraint that makes sure that if there are social distancing violations,
# they take up at most 20% of the dining room
for k in range(K):
    m.addConstr(e[k], GRB.LESS_EQUAL, l * w * 0.2)
    
# defining the constraint that makes sure that each party only occupies one table
for k in range(K):
    for i in range(I[k]):
        m.addConstr(x[i,k], GRB.LESS_EQUAL, 1)

# goal constraint of seating everyone in the restaurant
for k in range(K):
    total = 0
    seated = 0
    for i in range(I[k]):
        total += party_size[i,k]
        seated += party_size[i,k]*x[i,k]
    m.addConstr(seated + u[k], GRB.EQUAL, total)

        
        
# for k in range(K):
#     m.addConstr(s[k], GRB.LESS_EQUAL, l*w*0.2)
    
m.update()

l=15, w=15


#### Step 8: Solve the model

In [9]:
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 180 rows, 180 columns and 498 nonzeros
Model fingerprint: 0x420f4731
Variable types: 0 continuous, 180 integer (159 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 60100.000000
Presolve removed 179 rows and 174 columns
Presolve time: 0.02s
Presolved: 1 rows, 6 columns, 6 nonzeros
Found heuristic solution: objective 13440.000000
Variable types: 0 continuous, 6 integer (1 binary)

Root relaxation: objective 1.222000e+04, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 12220.0000    0    1 13440.0000 12220.0000  9.08%     -    0s
H    0     0                    12320.000000 12220.0000  0.81%     -    0s
     0     0 12

#### Step 8: Print variable values  (The Messy Way)

In [10]:
print("Printing result\n")
print('\nProfit: %g' % m.objVal) #gets the objective function value
print('\nSOLUTION')
for myVars in m.getVars():
    #print(myVars)
    print('%s %g' % (myVars.varName, myVars.x))

Printing result


Profit: 12320

SOLUTION
x_00 1
x_10 1
x_20 1
x_30 1
x_40 1
x_50 1
x_60 1
x_70 1
x_80 1
x_90 1
x_100 1
x_110 1
x_120 1
x_130 1
x_140 1
x_01 1
x_11 1
x_21 1
x_31 1
x_41 1
x_51 1
x_61 0
x_71 0
x_81 1
x_91 1
x_101 1
x_111 1
x_121 1
x_131 1
x_141 1
x_151 1
x_161 1
x_171 1
x_181 1
x_191 1
x_201 1
x_211 1
x_221 1
x_02 1
x_12 1
x_22 1
x_32 1
x_42 1
x_52 1
x_62 1
x_72 0
x_82 1
x_92 1
x_102 1
x_112 1
x_122 1
x_132 1
x_142 1
x_152 1
x_162 1
x_172 1
x_182 1
x_192 1
x_202 1
x_212 1
x_03 1
x_13 1
x_23 1
x_33 1
x_43 1
x_53 1
x_63 1
x_73 1
x_83 1
x_93 1
x_103 1
x_113 1
x_123 1
x_133 1
x_04 -0
x_14 1
x_24 0
x_34 1
x_44 1
x_54 0
x_64 0
x_74 1
x_84 0
x_94 1
x_104 1
x_114 0
x_124 1
x_134 1
x_144 1
x_154 1
x_164 1
x_174 1
x_184 0
x_194 1
x_204 0
x_214 1
x_224 0
x_234 0
x_244 -0
x_254 0
x_264 1
x_274 1
x_284 0
x_294 0
x_304 1
x_314 1
x_324 0
x_334 1
x_05 1
x_15 0
x_25 0
x_35 0
x_45 0
x_55 0
x_65 1
x_75 0
x_85 0
x_95 1
x_105 0
x_115 1
x_125 1
x_135 1
x_145 1
x_155 1
x_165 0
x_175 1
x_185 0


In [11]:
print('Model Summary\n\nDeviation from goals: %g' % m.objVal) #gets the objective function value

for k in range(K):
    res = ""
    if b[k].x != 1:
        res = "not"
    
    print(f"\nTimeslot {k+1}:\n")
    sum = 0
    for i in range(I[k]):
        if(x[i,k].x==1):
            sum += 1
            print(f"party {i+1} of size {party_size[i,k]} was seated")
        else:
            print(f"party {i+1} of size {party_size[i,k]} was not seated")

    print(f"\nPhysical distancing guideline {res} broken")
    print(f"Total number of parties seated during timeslot {k+1}: \t{sum} = {sum/I[k]*100:.2f}%")
    if b[k].x == 1:
        print(f"Amount of overoccupied space: {e[k].x}m^2\n")
    else:
        print(f"Amount of unoccupied space that can be allocated toward more customers: {s[k].x}m^2\n")

Model Summary

Deviation from goals: 12320

Timeslot 1:

party 1 of size 6 was seated
party 2 of size 6 was seated
party 3 of size 5 was seated
party 4 of size 4 was seated
party 5 of size 2 was seated
party 6 of size 5 was seated
party 7 of size 6 was seated
party 8 of size 2 was seated
party 9 of size 4 was seated
party 10 of size 5 was seated
party 11 of size 4 was seated
party 12 of size 2 was seated
party 13 of size 2 was seated
party 14 of size 4 was seated
party 15 of size 6 was seated

Physical distancing guideline not broken
Total number of parties seated during timeslot 1: 	15 = 100.00%
Amount of unoccupied space that can be allocated toward more customers: 36.0m^2


Timeslot 2:

party 1 of size 5 was seated
party 2 of size 4 was seated
party 3 of size 4 was seated
party 4 of size 3 was seated
party 5 of size 2 was seated
party 6 of size 3 was seated
party 7 of size 3 was not seated
party 8 of size 3 was not seated
party 9 of size 3 was seated
party 10 of size 5 was seated
pa